In [1]:
# univariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
import pandas as pd
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


In [21]:
# split the dataset into train/test sets
def split_dataset(data):
    threshold=int(0.7*len(data))
    train, test = data[1:threshold-98], data[threshold:-1]

    print('Train len=', len(train))
    print('Test len=', len(test))

    # restructure into windows of weekly data
    train = array(split(train, 200))
    test = array(split(test, 30))

    return train, test

In [3]:
def to_supervised(dataset):
    threshold=int(0.7*len(dataset))

    X, y = list(), list()

    for column in dataset.iloc[:, :-1]:
        x=dataset[column].iloc[0:threshold].values
        x=x.reshape((len(x), 1))
        X.append(x)
        y.append(dataset[column].iloc[threshold:-1].values)

    return array(x), array(y)


In [10]:
def to_supervised_v2(train, n_input, n_out=7):
    # flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

In [26]:
def build_model(train_x, train_y):
    verbose, epochs, batch_size = 0, 20, 16

    n_timesteps=train_x.shape[1]
    n_features=train_x.shape[0]
    n_output=train_y.shape[1]

    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_output))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')

    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    print('X=', train_x.shape[0], 'and', train_x.shape[1])
    print('Y=', train_y.shape[0], 'and', train_y.shape[1])

    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

    return model
    

In [5]:
dataset=pd.read_excel('../Data/ContinuousMotion_Data_Horizontalsetup.xlsx', sheet_name = "Sheet1")

In [23]:
train, test = split_dataset(dataset.values)

Train len= 31400
Test len= 13500


In [24]:
train_x, train_y=to_supervised_v2(train, n_input=30)

In [27]:
model=build_model(train_x, train_y)

X= 31364 and 30
Y= 31364 and 7


2022-12-14 13:21:21.081785: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb)

In [9]:
dataset.values.shape[2]

IndexError: tuple index out of range

In [29]:
# data = 1 2 3 4 5 6 7 8 9 10
# train = 1 2 3 4 5 6 
# test = 7 8 9 10

# train_x = 1 2 3 4
# train_y = 5 6

# reading data
# split into train - test 
# split train into train_x train_y
# algorithm input --> train_x train_y

# prediction of algorithm --> test 




SyntaxError: invalid syntax. Perhaps you forgot a comma? (1280865763.py, line 1)